In [2]:
!pip install gensim
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 22.2 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import gensim
import pandas as pd

In [4]:
%cd /content/drive/MyDrive/Colab Notebooks/NLP/word2vec_gensim

/content/drive/MyDrive/Colab Notebooks/NLP/word2vec_gensim


In [5]:
%pwd

'/content/drive/MyDrive/Colab Notebooks/NLP/word2vec_gensim'

In [6]:
# !wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz

--2024-02-21 03:13:37--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45409631 (43M) [application/x-gzip]
Saving to: ‘reviews_Cell_Phones_and_Accessories_5.json.gz’

reviews_Cell_Phones 100%[===================>]  43.31M  26.4MB/s    in 1.6s    

2024-02-21 03:13:39 (26.4 MB/s) - ‘reviews_Cell_Phones_and_Accessories_5.json.gz’ saved [45409631/45409631]



In [10]:
# !gzip -d reviews_Cell_Phones_and_Accessories_5.json.gz

In [6]:
df = pd.read_json('reviews_Cell_Phones_and_Accessories_5.json', lines=True)

In [7]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [8]:
df.shape

(194439, 9)

In [9]:
df.reviewText[0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

In [10]:
gensim.utils.simple_preprocess(df.reviewText[0])

['they',
 'look',
 'good',
 'and',
 'stick',
 'good',
 'just',
 'don',
 'like',
 'the',
 'rounded',
 'shape',
 'because',
 'was',
 'always',
 'bumping',
 'it',
 'and',
 'siri',
 'kept',
 'popping',
 'up',
 'and',
 'it',
 'was',
 'irritating',
 'just',
 'won',
 'buy',
 'product',
 'like',
 'this',
 'again']

In [11]:
review_text = df.reviewText.apply(gensim.utils.simple_preprocess)
review_text

0         [they, look, good, and, stick, good, just, don...
1         [these, stickers, work, like, the, review, say...
2         [these, are, awesome, and, make, my, phone, lo...
3         [item, arrived, in, great, time, and, was, in,...
4         [awesome, stays, on, and, looks, great, can, b...
                                ...                        
194434    [works, great, just, like, my, original, one, ...
194435    [great, product, great, packaging, high, quali...
194436    [this, is, great, cable, just, as, good, as, t...
194437    [really, like, it, becasue, it, works, well, w...
194438    [product, as, described, have, wasted, lot, of...
Name: reviewText, Length: 194439, dtype: object

In [12]:
model = gensim.models.Word2Vec(
    window = 10, #ngram
    min_count=2, # at least 2 word in sentence
    workers = 4,
)

In [13]:
model.build_vocab(review_text, progress_per=1000)

In [14]:
model.epochs

5

In [15]:
model.train(review_text, total_examples= model.corpus_count, epochs = model.epochs)

(61509240, 83868975)

In [16]:
model.save("./word2vec.model")

In [18]:
model.wv.most_similar("bad")

[('terrible', 0.6729329824447632),
 ('shabby', 0.6595962047576904),
 ('good', 0.587773859500885),
 ('horrible', 0.5719099044799805),
 ('okay', 0.5500782132148743),
 ('stellar', 0.5288063287734985),
 ('disappointing', 0.5254762768745422),
 ('poor', 0.5214034914970398),
 ('ok', 0.5144558548927307),
 ('crappy', 0.5105260610580444)]

In [19]:
model.wv.similarity("bad", "good")

0.58777386

In [20]:
model.wv.similarity("great", "product")

-0.032611568

In [21]:
model.wv.similarity("awesome", "good")

0.59563667